# Movie Review Explanations


We will use an SKLearn classifier built on [movie sentiment data](http://www.cs.cornell.edu/people/pabo/movie%2Dreview%2Ddata/) which predicts positive or negative sentiment for review text. 

The Kfserving resource provides:
   * A pretrained sklearn model stored on a Google bucket
   * A Text [Seldon Alibi](https://github.com/SeldonIO/alibi) Explainer. See the [Alibi Docs](https://docs.seldon.io/projects/alibi/en/stable/) for further details.

In [ ]:
!pygmentize moviesentiment.yaml

In [ ]:
!kubectl apply -f moviesentiment.yaml

In [ ]:
CLUSTER_IPS=!(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
CLUSTER_IP=CLUSTER_IPS[0]
print(CLUSTER_IP)

In [ ]:
SERVICE_HOSTNAMES=!(kubectl get inferenceservice moviesentiment -o jsonpath='{.status.url}' | cut -d "/" -f 3)
SERVICE_HOSTNAME=SERVICE_HOSTNAMES[0]
print(SERVICE_HOSTNAME)

In [ ]:
import sys
sys.path.append('../')
from alibi_helper import *

In [ ]:
from alibi.datasets import fetch_movie_sentiment
movies = fetch_movie_sentiment()

In [ ]:
idxNeg = 37
idxPos = 5227
for idx in [idxNeg,idxPos]:
    print(movies.data[idx])
    show_prediction(predict(movies.data[idx:idx+1],'moviesentiment',movies,SERVICE_HOSTNAME,CLUSTER_IP))

## Get Explanation for Negative Prediction

In [ ]:
exp = explain(movies.data[idxNeg:idxNeg+1],"moviesentiment",SERVICE_HOSTNAME,CLUSTER_IP)

In [ ]:
show_anchors(exp['data']['anchor'])

Show precision. How likely predictions using the Anchor features would produce the same result.

In [ ]:
show_bar([exp['data']['precision']],[''],"Precision")
show_bar([exp['data']['coverage']],[''],"Coverage")

In [ ]:
show_feature_coverage(exp['data'])

In [ ]:
show_examples(exp['data'],0,movies)

In [ ]:
show_examples(exp['data'],0,movies,False)

## Get Explanation for Positive Example

In [ ]:
exp = explain(movies.data[idxPos:idxPos+1],"moviesentiment",SERVICE_HOSTNAME,CLUSTER_IP)

In [ ]:
show_anchors(exp['data']['anchor'])

Show precision. How likely predictions using the Anchor features would produce the same result.

In [ ]:
show_bar([exp['data']['precision']],[''],"Precision")
show_bar([exp['data']['coverage']],[''],"Coverage")

In [ ]:
show_feature_coverage(exp['data'])

In [ ]:
show_examples(exp['data'],0,movies)

In [ ]:
show_examples(exp['data'],0,movies,False)

## Teardown

In [ ]:
!kubectl delete -f moviesentiment.yaml